In [23]:
import os
from neo4j import GraphDatabase
from dotenv import load_dotenv

load_dotenv()

graph = GraphDatabase.driver(
	os.environ['NEO4J_URI'],
	auth=(os.environ['NEO4J_USER'], os.environ['NEO4J_PWD'])
)
database = os.environ['NEO4J_DATABASE']

with graph.session(database=database) as session:
	records = session.execute_read(
		lambda tx: tx.run('MATCH (b:Bird)-[:HAS_FACT]->(d:Fact) ORDER BY RAND() RETURN b.name, d.text LIMIT 20').data()
	)
records

[{'b.name': 'Violet Crow',
  'd.text': 'Until recently considered conspecific with C. enca, but molecular evidence (1) suggests that it is unrelated; differs from taxa in that species (which see) in its barely visible but (on specimens) well-defined black hood contrasting with deep blue-black body (2); slightly shorter bill than other taxa (1) except C. e. sierramadrensis, which has a much smaller wing and tail (at least 2); and two highly distinct vocalizations, a short high “rah” (at least 2) and a nasal mewing “nyeh” (at least 2) (2). Monotypic.'},
 {'b.name': "Wallace's Hanging-Parrot",
  'd.text': 'Primary semi-evergreen forest, chiefly 800–1000 m but with records down to 450 m.'},
 {'b.name': "Forster's Tern",
  'd.text': "Forster's Tern is the only tern restricted almost entirely to North America throughout the year. Its highest breeding numbers are found around south-central Manitoba, northern California—southern Oregon, and the Gulf Coast. Its centers of abundance in early win

In [26]:
from typing import (
	List
)
import ollama
from pydantic import BaseModel, Field, ValidationError

class Location(BaseModel):
	name: str = Field(
		description='the name of the location as described in the text. Please expand any abbreviations, e.g. "SW Africa" should become "Southwest Africa".'
	)
	continents: List[str] = Field(
		description='all the continents that the location belongs to.'
	)
	countries: List[str] = Field(
		description='all the countries that the location could exist within.'
	)
	regions: List[str] = Field(
		description='all the regions that could potentially be a part of the location. These can be more casual delineations, and do not need to be formal.'
	)
	geographic_landmarks: List[str] = Field(
		description='any physical landmarks such as islands, penninsulas, mountain ranges, etc. that are a part of this location. Unlike continents/countries/regions, no not include these unless they are mentioned explicitly in the text.'
	)
	

class LocationEntries(BaseModel):
    mentioned_locations: List[Location]


class ValidationEntry(BaseModel):
    satisfactory: bool


def parse_locations(location_text: str) -> LocationEntries:
	temp = 0
	while temp < 1:
		try:
			response = ollama.chat(
				model='llama3.1:8b',
				messages=[
					{
						'role': 'user',
						'content': (
							'You are a location extraction engine. '
							'Your goal is to return a JSON list of all the locations you are able to extract from a given piece of text. '
							'A "location" can be a range (e.g. SW Africa), a region, a physical landmark, or some other distinct location entity. '
							'This is the format you should use in your response:\n'
							'{\n'
							'\t"mentioned_locations": [\n'
							'\t\t{\n'
							'\t\t\t"name": str - the name of the location as described in the text. Please expand any abbreviations, e.g. "SW Africa" should become "Southwest Africa".\n'
							'\t\t\t"continents": list[str] - all the continents that the location belongs to.\n'
							'\t\t\t"countries": list[str] - all the countries that the location could exist within.\n'
							'\t\t\t"regions": list[str] - all the regions that could potentially be a part of the location. These can be more casual delineations, and do not need to be formal.\n'
							'\t\t\t"geographic_landmarks": list[str] - any physical landmarks such as islands, penninsulas, mountain ranges, etc. that are a part of this location. Unlike continents/countries/regions, no not include these unless they are mentioned explicitly in the text.\n'
							'\t\t}\n'
							'\t\t...more locations...\n'
							'\t]\n'
							'}\n'
							'If there are NO locations in the text, return "{ "mentioned_locations": [] }". '
							'ONLY return locations that are EXPLICITLY MENTIONED, no entries like "global" or "worldwide".\n'
							f'This is the text to extract locations from:\n\n'
							f'"{location_text}"\n\n'
							'ONLY return the JSON, DO NOT EXPLAIN.'
						)
					}
				],
				format=LocationEntries.model_json_schema(), # <--- Native support
				options={'temperature': temp}#, 'num_ctx': 8192, 'num_predict': 512}
			)['message']['content']

			return LocationEntries.model_validate_json(response)

		except ValidationError:
			temp += 0.1

#'- Refers to an actual verifiable location.\n'
#'- The attributes in the JSON are related to each other. E.g. the "continent" being "Africa" and the "country" being the "United States" would fail this criteria.\n\n'

def validate_location(location: Location) -> bool:
	temp = 0
	while temp < 1:
		try:
			val_response = ollama.chat(
				model='llama3.1:8b',
				messages=[
					{
						'role': 'user',
						'content': (
							'Evaluate the following JSON that corresponds to a single location entry. Your goal is to answer if the location data meets the following criteria:\n'
							'- Location is SPECIFIC. Not "global", "worldwide", or anything that is NOT a SPECIFIC LOCATION ON THE MAP.\n\n'
							'- Does not refer to a library or other man-made building. Only geographic features.\n'
							f'JSON to evaluate: "{dict(location)}"'
						)
					}
				],
				format=ValidationEntry.model_json_schema(), # <--- Native support
				options={'temperature': 0}#, 'num_ctx': 8192, 'num_predict': 512}
			)['message']['content']

			return ValidationEntry.model_validate_json(val_response).satisfactory

		except ValidationError:
			temp += 0.1

 
for record in records:
	location_text = record['d.text']
	print(f'Location Text: "{location_text}"')
	locations = parse_locations(location_text)
	print(f'Found {len(locations.mentioned_locations)} locations\n')
	

	for ind, location in enumerate(locations.mentioned_locations):
		print(f'Loc {ind+1}: {location}')
		is_valid = validate_location(location)
		print(f'IS VALID: {is_valid}')

KeyError: 'd.text'

In [28]:
from typing import (
	Union,
    Dict
)
import os
from neo4j import GraphDatabase
from tqdm.notebook import tqdm
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(name)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    force=True,  # reset handlers if notebook re-runs
)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)

logger = logging.getLogger("notebook")


from dotenv import load_dotenv

load_dotenv()

graph = GraphDatabase.driver(
	os.environ['NEO4J_URI'],
	auth=(os.environ['NEO4J_USER'], os.environ['NEO4J_PWD'])
)
database = os.environ['NEO4J_DATABASE']

#idx = -1
idx = 8286
batch_size = 200
num_processed = 0


def get_batch(start_idx: int) -> Dict[str, str]:
	return session.execute_read(
		lambda tx: tx.run('''
			MATCH (f:Fact)
			WHERE id(f) > $idx
			ORDER BY id(f)
			RETURN id(f) as f_id, f.text as f_text
			LIMIT $batch_size
		''', idx=start_idx, batch_size=batch_size).data()
	)


def parse_locations_from_record(record):
	out = []
	locations = parse_locations(record['f_text'])
	for loc in locations.mentioned_locations:
		if validate_location(loc):
			out.append({"f_id": record["f_id"], **dict(loc)})
	return out


with graph.session(database=database) as session:
	records = get_batch(idx)

while records:
	with graph.session(database=database) as session:
		logger.info(f'BATCH {num_processed}-{num_processed+batch_size}')
		batch_data = [
			{ 'f_id': record['f_id'] }
			for record in records
		]

		batch_locations = []

		for record in tqdm(records):
			locations = parse_locations(record['f_text'])
			for loc in locations.mentioned_locations:
				if validate_location(loc):
					batch_locations.append({"f_id": record["f_id"], **dict(loc)})

		logger.info('WRITING TO DB...')

		session.execute_write(
			lambda tx: tx.run('''
				UNWIND $batch_locations AS row

				MATCH (f:Fact) WHERE id(f) = row.f_id

				CREATE (l:Location {name: row.name}) 
				CREATE (l)-[:MENTIONED_IN]->(f)

				FOREACH (continent_name IN row.continents |
					MERGE (cn:Continent {name: continent_name})
					CREATE (l)-[:IS_IN]->(cn)
				)

				FOREACH (country_name IN row.countries |
					MERGE (c:Country {name: country_name})
					CREATE (l)-[:IS_IN]->(c)
				)

				FOREACH (region_name IN row.regions |
					MERGE (r:Region {name: region_name})
					CREATE (l)-[:IS_IN]->(r)
				)

				FOREACH (geo_land_name in row.geographic_landmarks|
					MERGE (g:GeoLandmark {name: geo_land_name})
					CREATE (l)-[:IS_IN]->(g)
				)
			''', batch_locations=batch_locations)
		)

		num_processed += len(records)
		idx = max(records, key=lambda r: r['f_id'])['f_id']
		logger.info(f'Processed {num_processed} Entries, MAX IDX {idx}')
		
		records = get_batch(idx)

2025-12-10 18:51:15 WARNING neo4j.notifications: Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 10, offset: 28} for query: '\n\t\t\tMATCH (f:Fact)\n\t\t\tWHERE id(f) > $idx\n\t\t\tORDER BY id(f)\n\t\t\tRETURN id(f) as f_id, f.text as f_text\n\t\t\tLIMIT $batch_size\n\t\t'
2025-12-10 18:51:15 WARNING neo4j.notifications: Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 5, column: 11, offset: 69} for query: '\n\t\t\tMATCH (f:Fact)\n\t\t\tWHERE id(f) > $idx\n\t\t\tORDER BY id(f

  0%|          | 0/200 [00:00<?, ?it/s]

2025-12-10 18:59:31 INFO notebook: WRITING TO DB...
2025-12-10 18:59:31 WARNING neo4j.notifications: Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 4, column: 26, offset: 62} for query: '\n\t\t\t\tUNWIND $batch_locations AS row\n\n\t\t\t\tMATCH (f:Fact) WHERE id(f) = row.f_id\n\n\t\t\t\tCREATE (l:Location {name: row.name}) \n\t\t\t\tCREATE (l)-[:MENTIONED_IN]->(f)\n\n\t\t\t\tFOREACH (continent_name IN row.continents |\n\t\t\t\t\tMERGE (cn:Continent {name: continent_name})\n\t\t\t\t\tCREATE (l)-[:IS_IN]->(cn)\n\t\t\t\t)\n\n\t\t\t\tFOREACH (country_name IN row.countries |\n\t\t\t\t\tMERGE (c:Country {name: country_name})\n\t\t\t\t\tCREATE (l)-[:IS_IN]->(c)\n\t\t\t\t)\n\n\t\t\t\tFOREACH (region_name IN row.regions |\n\t\t\t\t\tMERGE (r:R

  0%|          | 0/200 [00:00<?, ?it/s]

2025-12-10 19:07:14 INFO notebook: WRITING TO DB...
2025-12-10 19:07:14 WARNING neo4j.notifications: Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 4, column: 26, offset: 62} for query: '\n\t\t\t\tUNWIND $batch_locations AS row\n\n\t\t\t\tMATCH (f:Fact) WHERE id(f) = row.f_id\n\n\t\t\t\tCREATE (l:Location {name: row.name}) \n\t\t\t\tCREATE (l)-[:MENTIONED_IN]->(f)\n\n\t\t\t\tFOREACH (continent_name IN row.continents |\n\t\t\t\t\tMERGE (cn:Continent {name: continent_name})\n\t\t\t\t\tCREATE (l)-[:IS_IN]->(cn)\n\t\t\t\t)\n\n\t\t\t\tFOREACH (country_name IN row.countries |\n\t\t\t\t\tMERGE (c:Country {name: country_name})\n\t\t\t\t\tCREATE (l)-[:IS_IN]->(c)\n\t\t\t\t)\n\n\t\t\t\tFOREACH (region_name IN row.regions |\n\t\t\t\t\tMERGE (r:R

...

2025-12-13 04:24:22 INFO notebook: WRITING TO DB...
2025-12-13 04:24:23 WARNING neo4j.notifications: Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 4, column: 26, offset: 62} for query: '\n\t\t\t\tUNWIND $batch_locations AS row\n\n\t\t\t\tMATCH (f:Fact) WHERE id(f) = row.f_id\n\n\t\t\t\tCREATE (l:Location {name: row.name}) \n\t\t\t\tCREATE (l)-[:MENTIONED_IN]->(f)\n\n\t\t\t\tFOREACH (continent_name IN row.continents |\n\t\t\t\t\tMERGE (cn:Continent {name: continent_name})\n\t\t\t\t\tCREATE (l)-[:IS_IN]->(cn)\n\t\t\t\t)\n\n\t\t\t\tFOREACH (country_name IN row.countries |\n\t\t\t\t\tMERGE (c:Country {name: country_name})\n\t\t\t\t\tCREATE (l)-[:IS_IN]->(c)\n\t\t\t\t)\n\n\t\t\t\tFOREACH (region_name IN row.regions |\n\t\t\t\t\tMERGE (r:R

  0%|          | 0/200 [00:00<?, ?it/s]

2025-12-13 04:28:45 INFO notebook: WRITING TO DB...
2025-12-13 04:28:47 WARNING neo4j.notifications: Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 4, column: 26, offset: 62} for query: '\n\t\t\t\tUNWIND $batch_locations AS row\n\n\t\t\t\tMATCH (f:Fact) WHERE id(f) = row.f_id\n\n\t\t\t\tCREATE (l:Location {name: row.name}) \n\t\t\t\tCREATE (l)-[:MENTIONED_IN]->(f)\n\n\t\t\t\tFOREACH (continent_name IN row.continents |\n\t\t\t\t\tMERGE (cn:Continent {name: continent_name})\n\t\t\t\t\tCREATE (l)-[:IS_IN]->(cn)\n\t\t\t\t)\n\n\t\t\t\tFOREACH (country_name IN row.countries |\n\t\t\t\t\tMERGE (c:Country {name: country_name})\n\t\t\t\t\tCREATE (l)-[:IS_IN]->(c)\n\t\t\t\t)\n\n\t\t\t\tFOREACH (region_name IN row.regions |\n\t\t\t\t\tMERGE (r:R

  0%|          | 0/92 [00:00<?, ?it/s]

2025-12-13 04:31:15 INFO notebook: WRITING TO DB...
2025-12-13 04:31:16 WARNING neo4j.notifications: Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 4, column: 26, offset: 62} for query: '\n\t\t\t\tUNWIND $batch_locations AS row\n\n\t\t\t\tMATCH (f:Fact) WHERE id(f) = row.f_id\n\n\t\t\t\tCREATE (l:Location {name: row.name}) \n\t\t\t\tCREATE (l)-[:MENTIONED_IN]->(f)\n\n\t\t\t\tFOREACH (continent_name IN row.continents |\n\t\t\t\t\tMERGE (cn:Continent {name: continent_name})\n\t\t\t\t\tCREATE (l)-[:IS_IN]->(cn)\n\t\t\t\t)\n\n\t\t\t\tFOREACH (country_name IN row.countries |\n\t\t\t\t\tMERGE (c:Country {name: country_name})\n\t\t\t\t\tCREATE (l)-[:IS_IN]->(c)\n\t\t\t\t)\n\n\t\t\t\tFOREACH (region_name IN row.regions |\n\t\t\t\t\tMERGE (r:R